In [1]:
# 读取数据
import json
nlp_retrieval= json.load(open(file='./大数据导论知识问答/anwser_NLP.json', mode='r', encoding='utf-8'))
bm25_retrieval= json.load(open(file='./大数据导论知识问答/BM25_answer.json', mode='r', encoding='utf-8'))

In [2]:
for bm25_ref in bm25_retrieval:
    for nlp_ref in nlp_retrieval:
        if bm25_ref['reference'] == nlp_ref['reference']:
            pass
        else:
            bm25_ref['reference'] = None
#保存数据
json.dump(bm25_retrieval, open(file='./大数据导论知识问答/answers_fusion.json', mode='w', encoding='utf-8'), ensure_ascii=False)

# 加载模型

In [3]:
# 加载重排序模型
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('../model/rerank-model/bge-reranker-v2-m3/')
rerank_model = AutoModelForSequenceClassification.from_pretrained('../model/rerank-model/bge-reranker-v2-m3/')
rerank_model.cuda()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_f

# 写入数据

In [4]:
import jieba, json, pdfplumber
questions = json.load(open("./大数据导论知识问答/question_list.json"))

pdf = pdfplumber.open("./大数据导论知识问答/大数据导论.pdf")
pdf_content = []
for page_idx in range(len(pdf.pages)):
    pdf_content.append({
        'page': 'page_' + str(page_idx + 1),
        'content': pdf.pages[page_idx].extract_text()
    })

# rerank

In [5]:
from rank_bm25 import BM25Okapi
pdf_content_words = [jieba.lcut(x['content']) for x in pdf_content]
bm25 = BM25Okapi(pdf_content_words)

for query_idx in range(len(questions)):
		# 首先进行BM25检索
    doc_scores = bm25.get_scores(jieba.lcut(questions[query_idx]["question"]))
    max_score_page_idxs = doc_scores.argsort()[-3:]
		
		# top3进行重排序
    pairs = []
    for idx in max_score_page_idxs:
        pairs.append([questions[query_idx]["question"], pdf_content[idx]['content']])

    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        inputs = {key: inputs[key].cuda() for key in inputs.keys()}
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()

    max_score_page_idx = max_score_page_idxs[scores.cpu().numpy().argmax()]
    questions[query_idx]['reference'] = 'page_' + str(max_score_page_idx + 1)

with open('submit.json', 'w', encoding='utf8') as up:
    json.dump(questions, up, ensure_ascii=False, indent=4)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\FUJIWA~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.753 seconds.
Prefix dict has been built successfully.
